In [11]:
from datasets import load_dataset

ds = load_dataset("sewon/ambig_qa", "light")

In [4]:
data = ds['train']
print(len(data))

10036


In [5]:
data[1]

{'id': '4790842463458965203',
 'question': 'Who played george washington in the john adams series?',
 'annotations': {'type': ['singleAnswer'],
  'answer': [['David Morse']],
  'qaPairs': [{'question': [], 'answer': []}]}}

In [5]:
import pandas as pd
import json

# --- 1. Load your data (if not already loaded) ---
# data = [json.loads(line) for line in open("ambignq_train_data.json")]

rows = []
for entry in data:
    ann = entry["annotations"]

    # Extract question type (string)
    ann_type = ann["type"][0] if isinstance(ann["type"], list) else ann["type"]

    # Extract qaPairs safely
    qa_pairs = ann.get("qaPairs", [])
    if len(qa_pairs) == 1 and isinstance(qa_pairs[0], list):
        qa_pairs = qa_pairs[0]

    # Extract single-answer list (only for singleAnswer entries)
    single_ans_list = ann.get("answer", [])
    single_ans_flat = []
    if isinstance(single_ans_list, list):
        for ans in single_ans_list:
            if isinstance(ans, list):
                single_ans_flat.extend(ans)
            elif isinstance(ans, str):
                single_ans_flat.append(ans)

    row = {
        "question": entry["question"],
        "type": ann_type,
        "qaPairs": qa_pairs,
        "single_answers": single_ans_flat if ann_type == "singleAnswer" else []
    }
    rows.append(row)

og_df = pd.DataFrame(rows)

# =========================================================
# 🔹 Extract Disambiguated Questions
# =========================================================
def extract_disambig_questions(qa_pairs):
    disambig = []
    if not isinstance(qa_pairs, list):
        return disambig
    for pair in qa_pairs:
        if not isinstance(pair, dict):
            continue
        qs = pair.get("question", [])
        if isinstance(qs, list):
            disambig.extend(qs)
        elif isinstance(qs, str):
            disambig.append(qs)
    return disambig


# =========================================================
# 🔹 Extract Disambiguated Answers
# =========================================================
def extract_disambig_answers(qa_pairs):
    answers = []
    if not isinstance(qa_pairs, list):
        return answers
    for pair in qa_pairs:
        if not isinstance(pair, dict):
            continue
        ans_list = pair.get("answer", [])
        if isinstance(ans_list, list):
            for a in ans_list:
                if isinstance(a, list):
                    answers.extend(a)
                elif isinstance(a, str):
                    answers.append(a)
    return answers


# =========================================================
# 🔹 Apply extraction to DataFrame
# =========================================================
og_df["disambig_questions"] = og_df["qaPairs"].apply(extract_disambig_questions)
og_df["disambig_answers"] = og_df["qaPairs"].apply(extract_disambig_answers)
og_df["is_ambiguous"] = og_df["type"].apply(lambda t: t == "multipleQAs")

# --- For singleAnswer entries, copy their answers from annotations ---
og_df["single_answers"] = og_df.apply(
    lambda r: r["single_answers"] if r["type"] == "singleAnswer" else [], axis=1
)

# =========================================================
# ✅ Sanity Check
# =========================================================
print(og_df[["question", "type", "disambig_questions", "disambig_answers", "single_answers", "is_ambiguous"]].head(5))


                                            question          type  \
0     When did the simpsons first air on television?   multipleQAs   
1  Who played george washington in the john adams...  singleAnswer   
2          What is the legal age of marriage in usa?   multipleQAs   
3   Who starred in barefoot in the park on broadway?   multipleQAs   
4      When did the manhattan project began and end?   multipleQAs   

                                  disambig_questions  \
0  [When did the Simpsons first air on television...   
1                                                 []   
2  [What is the legal age of marriage, without pa...   
3  [Who starred in barefoot in the park on broadw...   
4  [Based on the initial thoughts of the project,...   

                                    disambig_answers single_answers  \
0                [April 19, 1987, December 17, 1989]             []   
1                                                 []  [David Morse]   
2                   [18 years

In [7]:
og_df

,question,type,qaPairs,single_answers,disambig_questions,disambig_answers,is_ambiguous
0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,[],[When did the Simpsons first air on television...,"[April 19, 1987, December 17, 1989]",True
1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[David Morse],[],[],False
2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,[],"[What is the legal age of marriage, without pa...","[18 years of age, 18, 19, 21, 0]",True
3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,[],[Who starred in barefoot in the park on broadw...,"[Elizabeth Ashley, Kurt Kasznar, Mildred Natwi...",True
4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,[],"[Based on the initial thoughts of the project,...","[Began 1939, end 1946, Began 1942, end 1946]",True
...,...,...,...,...,...,...,...
10031,When do the summer holidays start for schools?,multipleQAs,[{'question': ['When do summer holidays start ...,[],[When do summer holidays start for schools in ...,"[end of June, between late-May and mid-June, B...",True
10032,Who is the band in the movie 10 things i hate ...,multipleQAs,[{'question': ['What band plays at Club Skunk ...,[],[What band plays at Club Skunk in the movie 10...,"[Letters to Cleo, Save Ferris, Letters to Cleo...",True
10033,Who was the last person in the uk to be executed?,singleAnswer,"[{'question': [], 'answer': []}]",[Gwynne Evans and Peter Allen],[],[],False
10034,Who does wonder woman end up with in the comics?,multipleQAs,[{'question': ['Who does wonder woman end up w...,[],[Who does wonder woman end up with in All Star...,"[General Steven Rockwell Trevor, Steve Trevor,...",True


Positive answers for disambiguous questions

In [8]:
full_facet_df=pd.read_csv('full_reasoning_df.csv')
full_facet_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'full_reasoning_df.csv'

In [ ]:
og_df

,question,type,qaPairs,single_answers,disambig_questions,disambig_answers,is_ambiguous
0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,[],[When did the Simpsons first air on television...,"[April 19, 1987, December 17, 1989]",True
1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[David Morse],[],[],False
2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,[],"[What is the legal age of marriage, without pa...","[18 years of age, 18, 19, 21, 0]",True
3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,[],[Who starred in barefoot in the park on broadw...,"[Elizabeth Ashley, Kurt Kasznar, Mildred Natwi...",True
4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,[],"[Based on the initial thoughts of the project,...","[Began 1939, end 1946, Began 1942, end 1946]",True
...,...,...,...,...,...,...,...
10031,When do the summer holidays start for schools?,multipleQAs,[{'question': ['When do summer holidays start ...,[],[When do summer holidays start for schools in ...,"[end of June, between late-May and mid-June, B...",True
10032,Who is the band in the movie 10 things i hate ...,multipleQAs,[{'question': ['What band plays at Club Skunk ...,[],[What band plays at Club Skunk in the movie 10...,"[Letters to Cleo, Save Ferris, Letters to Cleo...",True
10033,Who was the last person in the uk to be executed?,singleAnswer,"[{'question': [], 'answer': []}]",[Gwynne Evans and Peter Allen],[],[],False
10034,Who does wonder woman end up with in the comics?,multipleQAs,[{'question': ['Who does wonder woman end up w...,[],[Who does wonder woman end up with in All Star...,"[General Steven Rockwell Trevor, Steve Trevor,...",True


In [ ]:
pos_ans=full_facet_df.merge(og_df[['question','single_answers']], how='inner', on='question')
pos_ans

,Unnamed: 0,question,type,qaPairs,disambig_questions,is_ambiguous,facets,reasoning,single_answers
0,0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,['When did the Simpsons first air on televisio...,True,"['Type of Show', 'Platform']",The question is ambiguous because it does not ...,[]
1,1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,[David Morse]
2,2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,"['What is the legal age of marriage, without p...",True,"['State', 'Circumstances for Marriage']",The question is ambiguous because the legal ag...,[]
3,3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,['Who starred in barefoot in the park on broad...,True,['Character Name'],The question is ambiguous because it does not ...,[]
4,4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,['Based on the initial thoughts of the project...,True,['Time Frame Interpretation'],The question is ambiguous because it does not ...,[]
...,...,...,...,...,...,...,...,...,...
10031,10031,When do the summer holidays start for schools?,multipleQAs,[{'question': ['When do summer holidays start ...,['When do summer holidays start for schools in...,True,"['Geographical Region', 'Specific Location']",The question is ambiguous because it does not ...,[]
10032,10032,Who is the band in the movie 10 things i hate ...,multipleQAs,[{'question': ['What band plays at Club Skunk ...,['What band plays at Club Skunk in the movie 1...,True,"['Location', 'Event Type']",The question is ambiguous because it does not ...,[]
10033,10033,Who was the last person in the uk to be executed?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,[Gwynne Evans and Peter Allen]
10034,10034,Who does wonder woman end up with in the comics?,multipleQAs,[{'question': ['Who does wonder woman end up w...,['Who does wonder woman end up with in All Sta...,True,"['Story Universe', 'Publication Series']",The question is ambiguous because 'Wonder Woma...,[]


Generating positive responses for ambiguous questions

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# # Choose model
# model_name = "Qwen/Qwen2.5-7B-Instruct"

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)

# # Move to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",          # let Accelerate place layers
    torch_dtype=torch.bfloat16, # good precision for A100/T4
)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df=pos_ans.copy()
df=df[:990]
print(df.shape)
df.head()


(990, 9)


,Unnamed: 0,question,type,qaPairs,disambig_questions,is_ambiguous,facets,reasoning,single_answers
0,0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,['When did the Simpsons first air on televisio...,True,"['Type of Show', 'Platform']",The question is ambiguous because it does not ...,[]
1,1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,[David Morse]
2,2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,"['What is the legal age of marriage, without p...",True,"['State', 'Circumstances for Marriage']",The question is ambiguous because the legal ag...,[]
3,3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,['Who starred in barefoot in the park on broad...,True,['Character Name'],The question is ambiguous because it does not ...,[]
4,4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,['Based on the initial thoughts of the project...,True,['Time Frame Interpretation'],The question is ambiguous because it does not ...,[]


In [ ]:
# ==============================================================
# 🔹 IMPORTS & SETUP
# ==============================================================
import re
import ast
import gc
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---- Device Setup ----
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


# ==============================================================
# 🔹 Helper: Extract only [OUTPUT] content
# ==============================================================
def extract_output(text):
    """
    Extracts the model-generated clarifying question from between [OUTPUT] tags.
    """
    m = re.search(r"\[OUTPUT\](.*?)\[/OUTPUT\]", text, flags=re.DOTALL)
    if m:
        return m.group(1).strip()
    else:
        # fallback — return last few lines if no tags
        lines = text.strip().splitlines()
        return lines[-1].strip() if lines else text.strip()

# ==============================================================
# 🔹 Build Positive Clarifying Prompt
# ==============================================================
def build_positive_prompt(question, facets):
    facets_txt = ", ".join(facets) if facets else "N/A"
    #tax_txt = ", ".join(taxonomies) if taxonomies else "N/A"

    return f"""
You are a helpful assistant that clarifies ambiguous questions.

Your task: Ask a **single-turn clarifying question** that helps the user
specify their intent and resolve all the ambiguity. Use the provided facets (as clues about *what* is ambiguous)

---

**Ambiguous Question:**
{question}

**Facets (dimensions of ambiguity):**
{facets_txt}

---

Guidelines:
1. Ask one polite, natural question that helps clarify **all listed facets**.
2. Do **not** answer the question.
3. Keep your response short (1 sentence max).
4. Output the clarifying question only between [OUTPUT] and [/OUTPUT].

---

### Example 1
Ambiguous: "When did The Simpsons first air?"
Facets: Type of Show, Broadcast Region
Output: [OUTPUT]Could you clarify which type of Simpsons show you’re referring to, and in which region it aired?[/OUTPUT]

### Example 2
Ambiguous: "Who won the US Open?"
Facets: Sport Type, Year
Output: [OUTPUT]Do you mean the US Open in tennis or golf, and which year are you asking about?[/OUTPUT]

---

Now write your clarifying question.
Remember to wrap it exactly as [OUTPUT] ... [/OUTPUT].
"""

# ==============================================================
# 🔹 LLM Call Utility
# ==============================================================
def generate_text(prompt, temperature=0.5, max_new_tokens=100, debug=False):
    """
    Generates only the model's clarifying question, not the prompt.
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=3000).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=True
        )

    # Decode only the newly generated part, not the input prompt
    gen_tokens = outputs[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(gen_tokens, skip_special_tokens=True)

    if debug:
        print("\n=== RAW MODEL OUTPUT ===\n", text, "\n=========================\n")

    return extract_output(text)

# ==============================================================
# 🔹 MAIN LOOP — GENERATE POSITIVE RESPONSES ONLY
# ==============================================================
positive_responses = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating positive clarifying questions"):
    if not row.get("is_ambiguous", False):
        positive_responses.append("N/A")
        continue

    question = row["question"]

    # --- Parse facet_taxonomies safely ---
    facets_raw = row.get("facets", [])
    if isinstance(facets_raw, str):
        try:
            facets = ast.literal_eval(facets_raw)
        except Exception:
            facets = [facets_raw]
    elif isinstance(facets_raw, list):
        facets = facets_raw
    else:
        facets = []

    #taxonomies = [ft.get("taxonomy", "") for ft in raw_ft if isinstance(ft, dict)]

    try:
        pos_prompt = build_positive_prompt(question, facets)
        pos_out = generate_text(pos_prompt, temperature=0.6, debug=True)
        positive_responses.append(pos_out)
    except torch.cuda.OutOfMemoryError:
        print(f"\n⚠️ OOM at row {idx}, switching to CPU temporarily...")
        model.to("cpu")
        pos_out = generate_text(pos_prompt, temperature=0.6, debug=False)
        positive_responses.append(pos_out)
        model.to(device)
    finally:
        torch.cuda.empty_cache()
        gc.collect()

    # --- Print summary per question ---
    print(f"\n🟩 Row {idx}")
    print(f"Q: {question}")
    print(f"Facets: {facets}")
    #print(f"Taxonomies: {taxonomies}")
    print(f"✅ Clarifying Question: {pos_out}")
    print("------------------------------------------------------\n")

# ==============================================================
# 🔹 SAVE OUTPUT
# ==============================================================
df["positive_response"] = positive_responses
df.to_csv("positive_clarifying_responses.csv", index=False)
print("\n✅ Saved results to positive_clarifying_responses.csv")


Using device: cuda


Generating positive clarifying questions:   0%|          | 0/990 [00:00<?, ?it/s]


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the original TV series or a movie, and on which platform (e.g., network, streaming service) did it first air?[/OUTPUT] 



Generating positive clarifying questions:   0%|          | 1/990 [00:01<26:50,  1.63s/it]


🟩 Row 0
Q: When did the simpsons first air on television?
Facets: ['Type of Show', 'Platform']
✅ Clarifying Question: Are you asking about the original TV series or a movie, and on which platform (e.g., network, streaming service) did it first air?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]In which U.S. state and under what circumstances are you asking about the legal age of marriage?[/OUTPUT] 


🟩 Row 2
Q: What is the legal age of marriage in usa?
Facets: ['State', 'Circumstances for Marriage']
✅ Clarifying Question: In which U.S. state and under what circumstances are you asking about the legal age of marriage?
------------------------------------------------------



...


🟩 Row 979
Q: Where does rome's drinking water come from?
Facets: ['Time Period', 'Type of Water Source', 'Transportation Method']
✅ Clarifying Question: Are you asking about the historical sources of Rome's drinking water, the current sources, or both; what kind of water source (e.g., aqueducts, wells) are you interested in; and how do you want the information presented (e.g., through ancient records, modern methods)?
------------------------------------------------------



Generating positive clarifying questions:  99%|█████████▉| 984/990 [09:09<00:03,  1.77it/s]


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the type of alpha amylase, and in which specific part of the body it is found?[/OUTPUT] 


🟩 Row 983
Q: Where is alpha amylase found in the body?
Facets: ['Type of Alpha Amylase', 'Location in the Body']
✅ Clarifying Question: Are you asking about the type of alpha amylase, and in which specific part of the body it is found?
------------------------------------------------------



Generating positive clarifying questions: 100%|██████████| 990/990 [09:10<00:00,  1.80it/s]


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the total number of NBA Defensive Player of the Year awards won by an individual player, considering both consistency and continuity of the award over time?[/OUTPUT] 


🟩 Row 987
Q: Who has the most nba defensive player of the year awards?
Facets: ['Team Consistency', 'Award Continuity']
✅ Clarifying Question: Are you asking about the total number of NBA Defensive Player of the Year awards won by an individual player, considering both consistency and continuity of the award over time?
------------------------------------------------------


✅ Saved results to positive_clarifying_responses.csv


In [ ]:
df

,Unnamed: 0,question,type,qaPairs,disambig_questions,is_ambiguous,facets,reasoning,single_answers,positive_response
990,990,Kansas city chief player who is a doctor?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,"[Laurent Duvernay-Tardif CQ, Laurent Duvernay-...",N/A
991,991,When did the last civil war widow die?,multipleQAs,[{'question': ['When did the last civil war wi...,['When did the last civil war widow from the U...,True,['Side of Civil War'],The question is ambiguous because it does not ...,[],Could you specify which side of the Civil War ...
992,992,What does sho stand for on ford taurus?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,[Super High Output],N/A
993,993,Who won australia's next top model season 7?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,"[Montana Cox, Cox]",N/A
994,994,What is the climate in the gobi desert?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,"[a rain shadow desert, great extremes, combine...",N/A
...,...,...,...,...,...,...,...,...,...,...
10031,10031,When do the summer holidays start for schools?,multipleQAs,[{'question': ['When do summer holidays start ...,['When do summer holidays start for schools in...,True,"['Geographical Region', 'Specific Location']",The question is ambiguous because it does not ...,[],In which geographical region and specific loca...
10032,10032,Who is the band in the movie 10 things i hate ...,multipleQAs,[{'question': ['What band plays at Club Skunk ...,['What band plays at Club Skunk in the movie 1...,True,"['Location', 'Event Type']",The question is ambiguous because it does not ...,[],Are you asking about the band featured in the ...
10033,10033,Who was the last person in the uk to be executed?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,[Gwynne Evans and Peter Allen],N/A
10034,10034,Who does wonder woman end up with in the comics?,multipleQAs,[{'question': ['Who does wonder woman end up w...,['Who does wonder woman end up with in All Sta...,True,"['Story Universe', 'Publication Series']",The question is ambiguous because 'Wonder Woma...,[],Are you asking about Wonder Woman's relationsh...


## concat the first 1k df with the rest 9k df

In [ ]:
import pandas as pd
import numpy as np

first_half=pd.read_csv('positive_clarifying_responses-4.csv')
second_half=pd.read_csv('positive_clarifying_responses-3.csv')

print(first_half.shape)
print(second_half.shape)

(990, 10)
(9046, 10)


In [ ]:
first_half.tail()

,Unnamed: 0,question,type,qaPairs,disambig_questions,is_ambiguous,facets,reasoning,single_answers,positive_response
985,985,How many pages in a brave new world?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,['311'],NaN
986,986,When was the last time newcastle won the fa cup?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,['1954-55'],NaN
987,987,Who has the most nba defensive player of the y...,multipleQAs,[{'question': ['Who has the most nba defensive...,['Who has the most nba defensive player of the...,True,"['Team Consistency', 'Award Continuity']",The question is ambiguous because it does not ...,[],Are you asking about the total number of NBA D...
988,988,What are the components of the renal corpuscle?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,"[""glomerulus, Bowman's capsule""]",NaN
989,989,Who sang bird bird bird bird is the word?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it provi...,['The Trashmen'],NaN


In [ ]:
second_half.tail()

,Unnamed: 0,question,type,qaPairs,disambig_questions,is_ambiguous,facets,reasoning,single_answers,positive_response
9041,10031,When do the summer holidays start for schools?,multipleQAs,[{'question': ['When do summer holidays start ...,['When do summer holidays start for schools in...,True,"['Geographical Region', 'Specific Location']",The question is ambiguous because it does not ...,[],In which geographical region and specific loca...
9042,10032,Who is the band in the movie 10 things i hate ...,multipleQAs,[{'question': ['What band plays at Club Skunk ...,['What band plays at Club Skunk in the movie 1...,True,"['Location', 'Event Type']",The question is ambiguous because it does not ...,[],Are you asking about the band featured in the ...
9043,10033,Who was the last person in the uk to be executed?,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,['Gwynne Evans and Peter Allen'],NaN
9044,10034,Who does wonder woman end up with in the comics?,multipleQAs,[{'question': ['Who does wonder woman end up w...,['Who does wonder woman end up with in All Sta...,True,"['Story Universe', 'Publication Series']",The question is ambiguous because 'Wonder Woma...,[],Are you asking about Wonder Woman's relationsh...
9045,10035,When were the first pair of jordans released?,multipleQAs,[{'question': ['When were the first pair of Ai...,['When were the first pair of Air Jordans rele...,True,"['Release Target Audience', 'Release Scope']",The question is ambiguous because it does not ...,[],Could you specify whether you're asking about ...


In [ ]:
final_df = pd.concat([first_half, second_half], ignore_index=True)

# Step 2: Drop duplicates (keeps the first occurrence)
final_df = final_df.drop_duplicates(ignore_index=True)

# Check final shape
print(final_df.shape)

final_df.head()

(10036, 10)


,Unnamed: 0,question,type,qaPairs,disambig_questions,is_ambiguous,facets,reasoning,single_answers,positive_response
0,0,When did the simpsons first air on television?,multipleQAs,[{'question': ['When did the Simpsons first ai...,['When did the Simpsons first air on televisio...,True,"['Type of Show', 'Platform']",The question is ambiguous because it does not ...,[],Are you asking about the original TV series or...
1,1,Who played george washington in the john adams...,singleAnswer,"[{'question': [], 'answer': []}]",[],False,[],The question is not ambiguous because it speci...,['David Morse'],NaN
2,2,What is the legal age of marriage in usa?,multipleQAs,[{'question': ['What is the legal age of marri...,"['What is the legal age of marriage, without p...",True,"['State', 'Circumstances for Marriage']",The question is ambiguous because the legal ag...,[],In which U.S. state and under what circumstanc...
3,3,Who starred in barefoot in the park on broadway?,multipleQAs,[{'question': ['Who starred in barefoot in the...,['Who starred in barefoot in the park on broad...,True,['Character Name'],The question is ambiguous because it does not ...,[],Did you mean the character names or the actors...
4,4,When did the manhattan project began and end?,multipleQAs,[{'question': ['Based on the initial thoughts ...,['Based on the initial thoughts of the project...,True,['Time Frame Interpretation'],The question is ambiguous because it does not ...,[],In which specific year or time period are you ...


In [ ]:
final_df['action']=np.where(final_df['is_ambiguous']==True, 'Clarify', 'Answer')

In [3]:
final_df.to_csv('final_SFT_training_data.csv')

NameError: name 'final_df' is not defined

## SETTING UP DATA FOR TRAINING PHASE

In [9]:
final_df.columns

NameError: name 'final_df' is not defined

In [ ]:
final_df['positive_response']=final_df['positive_response'].fillna(final_df['single_answers'])

In [ ]:
final_sft_df=final_df[['question', 'is_ambiguous', 'facets', 'reasoning', 'positive_response', 'action']]
pd.set_option('display.max_colwidth', None)
final_sft_df.head()

,question,is_ambiguous,facets,reasoning,positive_response,action
0,When did the simpsons first air on television?,True,"['Type of Show', 'Platform']","The question is ambiguous because it does not specify whether 'The Simpsons' refers to the TV series or another type of show, and it does not clarify if the platform (e.g., network, streaming service) is relevant. Without this information, we cannot determine the exact date of its first broadcast.","Are you asking about the original TV series or a movie, and on which platform (e.g., network, streaming service) did it first air?",Clarify
1,Who played george washington in the john adams series?,False,[],"The question is not ambiguous because it specifies the exact role and series, providing enough detail to uniquely identify the actor who portrayed George Washington in the John Adams miniseries.",['David Morse'],Answer
2,What is the legal age of marriage in usa?,True,"['State', 'Circumstances for Marriage']","The question is ambiguous because the legal age of marriage can vary by state in the USA. Additionally, there might be different circumstances or exceptions that affect the age requirement, such as parental consent or a court order. Without specifying the state and the circumstances, the answer can differ significantly.",In which U.S. state and under what circumstances are you asking about the legal age of marriage?,Clarify
3,Who starred in barefoot in the park on broadway?,True,['Character Name'],"The question is ambiguous because it does not specify which character from the play ""Barefoot in the Park"" was starred in. Without knowing the character name, we cannot determine who played that specific role on Broadway.",Did you mean the character names or the actors who played them in the Broadway production of Barefoot in the Park?,Clarify
4,When did the manhattan project began and end?,True,['Time Frame Interpretation'],"The question is ambiguous because it does not specify which aspect of the Manhattan Project is being referred to—whether it's the start date, end date, or a broader time frame such as the duration. The term ""began and ended"" could imply either two separate dates or the entire period of the project.",In which specific year or time period are you interested in knowing about the Manhattan Project's start and end?,Clarify


In [ ]:
final_sft_df.to_csv('final_SFT_data_for_use.csv')

In [4]:
import pandas as pd
final_sft_df=pd.read_csv('final_SFT_data_for_use.csv')

In [ ]:
#results_primitive.to_csv('pos_amb_disambig_answers.csv')

### Pre-process the data to get it into proper json format

In [5]:
import json
import ast
import pandas as pd

df = final_sft_df.copy()

def clean_facets(facets):
    if isinstance(facets, list):
        return facets
    if isinstance(facets, str):
        try:
            val = ast.literal_eval(facets)
            return val if isinstance(val, list) else []
        except:
            return []
    return []

def clean_response(resp):
    if resp is None:
        return ""

    # If it's a list, return the first element
    if isinstance(resp, list):
        return resp[0] if len(resp) > 0 else ""

    # If it's a string of a list, extract content
    if isinstance(resp, str) and resp.startswith("[") and resp.endswith("]"):
        try:
            list_val = ast.literal_eval(resp)
            if isinstance(list_val, list) and len(list_val) > 0:
                return list_val[0]
        except:
            pass

    # Otherwise return clean string
    return resp.strip()


records = []

for _, row in df.iterrows():
    q = row["question"]
    is_amb = row["is_ambiguous"]
    reasoning = row["reasoning"]
    action = row["action"].strip()
    facets = clean_facets(row["facets"])

    # Clarify vs Answer
    response_raw = row["positive_response"]
    response = clean_response(response_raw)

    record = {
        "instruction": (
            "Decide whether the user's question is ambiguous. "
            "If ambiguous, explain the ambiguity, list facets, and generate a clarifying question. "
            "If not ambiguous, explain why and answer directly."
        ),
        "input": q,
        "output": {
            "action": action,
            "reasoning": reasoning,
            "facets": facets if is_amb else [],
            "response": response
        }
    }
    records.append(record)

with open("sft_data.jsonl", "w") as f:
    for r in records:
        f.write(json.dumps(r) + "\n")

print("⭐ CLEAN SFT dataset created: sft_data.jsonl")


⭐ CLEAN SFT dataset created: sft_data.jsonl


In [6]:
import json

with open("sft_data.jsonl") as f:
    for _ in range(5):
        print(json.loads(next(f)))
        print("------")


{'instruction': "Decide whether the user's question is ambiguous. If ambiguous, explain the ambiguity, list facets, and generate a clarifying question. If not ambiguous, explain why and answer directly.", 'input': 'When did the simpsons first air on television?', 'output': {'action': 'Clarify', 'reasoning': "The question is ambiguous because it does not specify whether 'The Simpsons' refers to the TV series or another type of show, and it does not clarify if the platform (e.g., network, streaming service) is relevant. Without this information, we cannot determine the exact date of its first broadcast.", 'facets': ['Type of Show', 'Platform'], 'response': 'Are you asking about the original TV series or a movie, and on which platform (e.g., network, streaming service) did it first air?'}}
------
{'instruction': "Decide whether the user's question is ambiguous. If ambiguous, explain the ambiguity, list facets, and generate a clarifying question. If not ambiguous, explain why and answer di

In [10]:
!pip install -q transformers==4.45.0 accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.3 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset

# Load full dataset
full_ds = load_dataset("json", data_files="sft_data.jsonl", split="train")

# Make a small test subset (e.g., 500 rows)
test_subset = full_ds.select(range(500))

print("Subset size:", len(test_subset))
test_subset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Subset size: 500


{'instruction': "Decide whether the user's question is ambiguous. If ambiguous, explain the ambiguity, list facets, and generate a clarifying question. If not ambiguous, explain why and answer directly.",
 'input': 'When did the simpsons first air on television?',
 'output': {'action': 'Clarify',
  'reasoning': "The question is ambiguous because it does not specify whether 'The Simpsons' refers to the TV series or another type of show, and it does not clarify if the platform (e.g., network, streaming service) is relevant. Without this information, we cannot determine the exact date of its first broadcast.",
  'facets': ['Type of Show', 'Platform'],
  'response': 'Are you asking about the original TV series or a movie, and on which platform (e.g., network, streaming service) did it first air?'}}

In [8]:
def format_example(example):
    question  = example["input"]
    action    = example["output"]["action"]
    reasoning = example["output"]["reasoning"]
    facets    = example["output"]["facets"]
    response  = example["output"]["response"]

    if isinstance(facets, list):
        facets_str = "[" + ", ".join(facets) + "]"
    else:
        facets_str = "[]"

    system_msg = (
        "You are a question understanding agent. For each user question, you must:\n"
        "1) Decide whether it is ambiguous.\n"
        "2) Explain your reasoning.\n"
        "3) List facets if ambiguous.\n"
        "4) Either ask a clarifying question or give a direct answer.\n\n"
        "Reply in the format:\n"
        "Action: Clarify|Answer\n"
        "Reasoning: ...\n"
        "Facets: [...]\n"
        "Response: ...\n"
    )

    formatted = (
        f"<System>\n{system_msg}\n</System>\n\n"
        f"<User>\n{question}\n</User>\n\n"
        f"<Assistant>\n"
        f"Action: {action}\n"
        f"Reasoning: {reasoning}\n"
        f"Facets: {facets_str}\n"
        f"Response: {response}\n"
        f"</Assistant>"
    )

    return {"text": formatted}

formatted_subset = test_subset.map(format_example, remove_columns=test_subset.column_names)
formatted_subset[0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'text': "<System>\nYou are a question understanding agent. For each user question, you must:\n1) Decide whether it is ambiguous.\n2) Explain your reasoning.\n3) List facets if ambiguous.\n4) Either ask a clarifying question or give a direct answer.\n\nReply in the format:\nAction: Clarify|Answer\nReasoning: ...\nFacets: [...]\nResponse: ...\n\n</System>\n\n<User>\nWhen did the simpsons first air on television?\n</User>\n\n<Assistant>\nAction: Clarify\nReasoning: The question is ambiguous because it does not specify whether 'The Simpsons' refers to the TV series or another type of show, and it does not clarify if the platform (e.g., network, streaming service) is relevant. Without this information, we cannot determine the exact date of its first broadcast.\nFacets: [Type of Show, Platform]\nResponse: Are you asking about the original TV series or a movie, and on which platform (e.g., network, streaming service) did it first air?\n</Assistant>"}

In [8]:
# First, completely clean the environment
!pip uninstall -y bitsandbytes transformers accelerate peft trl -q

# Install bitsandbytes from source (more reliable in Colab)
!pip install -q git+https://github.com/TimDettmers/bitsandbytes.git

# Install other dependencies
!pip install -q transformers==4.45.0
!pip install -q accelerate==0.34.0
!pip install -q peft==0.13.0
!pip install -q trl==0.12.0
!pip install -q datasets

# Verify bitsandbytes installation
!python -c "import bitsandbytes; print(f'bitsandbytes version: {bitsandbytes.__version__}')"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
bitsandbytes library load error: Configured CUDA binary not found at /usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda126.so
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/bitsandbytes/cextension.py", line 320, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/bitsandbytes/cextension.py", line 288, in get_native_library
    raise RuntimeError(f"Configured {BNB_BACKEND} binary not found at {cuda_binary_path}")
RuntimeError: Configured CUDA binary not found at /usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda126.so
bitsandbytes version: 0.49.0.dev0


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ModuleNotFoundError: Could not import module 'validate_bnb_backend_availability'. Are this object's requirements defined correctly?

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="clarifier_test_sft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,           # 1-epoch test
    learning_rate=2e-4,
    warmup_ratio=0.05,
    logging_steps=10,
    save_steps=200,
    bf16=True,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    report_to="none",
)


In [9]:
# First, check your CUDA version
!nvcc --version

# Uninstall current bitsandbytes
!pip uninstall -y bitsandbytes

# Install a stable pre-compiled version that works with Colab
!pip install bitsandbytes==0.43.1

# Restart runtime, then verify
!python -m bitsandbytes

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Found existing installation: bitsandbytes 0.49.0.dev0
Uninstalling bitsandbytes-0.49.0.dev0:
  Successfully uninstalled bitsandbytes-0.49.0.dev0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.8 MB/s eta 0:00:00


Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.12/dist-packages/bitsandbytes/__init__.py", line 6, in <module>
  File "/usr/local/lib/python3.12/dist-packages/bitsandbytes/research/__init__.py", line 1, in <module>
    from . import nn
  File "/usr/local/lib/python3.12/dist-packages/bitsandbytes/research/nn/__init__.py", line 1, in <module>
    from .modules import LinearFP8Global, LinearFP8Mixed
  File "/usr/local/lib/python3.12/dist-packages/bitsandbytes/research/nn/modules.py", line 3, in <module>
    import torch
  File "/usr/local/lib/python3.12/dist-packages/torch/__init__.py", line 2668, in <module>
    from torch import _meta_registrations
  File "/usr/local/lib/python3.12/dist-packages/torch/_meta_registrations.py", line 13, in <module>
    from torch._decomp import (
  File "/usr/local/

In [2]:
# Complete clean installation with all dependencies
!pip uninstall -y bitsandbytes transformers accelerate peft trl triton

# Install Triton first (required for bitsandbytes)
!pip install -q triton

# Install PyTorch with CUDA support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install bitsandbytes and other packages
!pip install -q bitsandbytes==0.43.1
!pip install -q transformers==4.45.0
!pip install -q accelerate==0.34.0
!pip install -q peft==0.13.0
!pip install -q trl==0.12.0
!pip install -q datasets

print("✅ Installation complete! RESTART RUNTIME NOW!")

Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1
Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 0.34.0
Uninstalling accelerate-0.34.0:
  Successfully uninstalled accelerate-0.34.0
Found existing installation: peft 0.13.0
Uninstalling peft-0.13.0:
  Successfully uninstalled peft-0.13.0
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
Found existing installation: triton 3.4.0
Uninstalling triton-3.4.0:
  Successfully uninstalled triton-3.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transf

In [11]:
# 1. Complete clean uninstall
!pip uninstall -y bitsandbytes transformers accelerate peft trl triton -q

# 2. Install ONLY what you need in correct order
!pip install -q transformers==4.45.0
!pip install -q accelerate==0.34.0
!pip install -q datasets
!pip install -q peft==0.13.0
!pip install -q trl==0.12.0

# 3. Skip bitsandbytes entirely - use alternative quantization
print("Installation done. RESTART RUNTIME NOW!")

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/lib/python3.12/shutil.py", line 847, in move
    os.rename(src, real_dst)
OSError: [Errno 18] Invalid cross-device link: '/usr/local/lib/python3.12/dist-packages/triton/' -> '/usr/local/lib/python3.12/dist-packages/~riton'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/req/req_uni

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# Option 1: No quantization (recommended for Colab)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

print("✅ Model loaded!")



`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:50<00:00, 12.67s/it]

✅ Model loaded!


In [12]:

# Format data
def format_text(row):
    return f"""### Question: {row['question']}
### Taxonomy: {row['taxonomy']}
### Facets: {row['facets']}"""

df['text'] = df.apply(format_text, axis=1)
dataset = Dataset.from_pandas(df[['text']])

# Load model
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Training args
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
)

# Max sequence length
max_seq_length = 2048

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=training_args,
    peft_config=lora_config,
)

# Train
trainer.train()

NameError: name 'dataset' is not defined

code to generate positive answers for ambiguous questions (with taxonomy) to be used later

In [14]:
# ==============================================================
# 🔹 IMPORTS & SETUP
# ==============================================================
import re
import ast
import gc
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---- Device Setup ----
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
df = df.head(500)
print(f"Processing {len(df)} rows")

# ==============================================================
# 🔹 Helper: Extract only [OUTPUT] content
# ==============================================================
def extract_output(text):
    """
    Extracts the model-generated clarifying question from between [OUTPUT] tags.
    """
    m = re.search(r"\[OUTPUT\](.*?)\[/OUTPUT\]", text, flags=re.DOTALL)
    if m:
        return m.group(1).strip()
    else:
        # fallback — return last few lines if no tags
        lines = text.strip().splitlines()
        return lines[-1].strip() if lines else text.strip()

# ==============================================================
# 🔹 Build Positive Clarifying Prompt
# ==============================================================
def build_positive_prompt(question, facets, taxonomies):
    facets_txt = ", ".join(facets) if facets else "N/A"
    tax_txt = ", ".join(taxonomies) if taxonomies else "N/A"

    return f"""
You are a helpful assistant that clarifies ambiguous questions.

Your task: Ask a **single-turn clarifying question** that helps the user
specify their intent and resolve all the ambiguity.

---

**Ambiguous Question:**
{question}

**Facets (dimensions of ambiguity):**
{facets_txt}

**Taxonomy Types (nature of ambiguity):**
{tax_txt}

---

Guidelines:
1. Ask one polite, natural question that helps clarify **all listed facets**.
2. Use the taxonomy types to understand what kind of clarification to ask.
3. Do **not** answer the question.
4. Keep your response short (1 sentence max).
5. Output the clarifying question only between [OUTPUT] and [/OUTPUT].

---

### Example 1
Ambiguous: "When did The Simpsons first air?"
Facets: Type of Show, Broadcast Region
Taxonomy: Part of Entity Reference, Geographical Dependency
Output: [OUTPUT]Could you clarify which type of Simpsons show you’re referring to, and in which region it aired?[/OUTPUT]

### Example 2
Ambiguous: "Who won the US Open?"
Facets: Sport Type, Year
Taxonomy: Entity Reference, Temporal Dependency
Output: [OUTPUT]Do you mean the US Open in tennis or golf, and which year are you asking about?[/OUTPUT]

---

Now write your clarifying question.
Remember to wrap it exactly as [OUTPUT] ... [/OUTPUT].
"""

# ==============================================================
# 🔹 LLM Call Utility
# ==============================================================
def generate_text(prompt, temperature=0.5, max_new_tokens=100, debug=False):
    """
    Generates only the model's clarifying question, not the prompt.
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=3000).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=True
        )

    # Decode only the newly generated part, not the input prompt
    gen_tokens = outputs[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(gen_tokens, skip_special_tokens=True)

    if debug:
        print("\n=== RAW MODEL OUTPUT ===\n", text, "\n=========================\n")

    return extract_output(text)

# ==============================================================
# 🔹 MAIN LOOP — GENERATE POSITIVE RESPONSES ONLY
# ==============================================================
positive_responses = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating positive clarifying questions"):
    if not row.get("is_ambiguous", False):
        positive_responses.append("N/A")
        continue

    question = row["question"]

    # --- Parse facet_taxonomies safely ---
    raw_ft = row.get("facet_taxonomies", [])
    if isinstance(raw_ft, str):
        try:
            raw_ft = ast.literal_eval(raw_ft)
        except Exception:
            raw_ft = []

    facets = [ft.get("facet", "") for ft in raw_ft if isinstance(ft, dict)]
    taxonomies = [ft.get("taxonomy", "") for ft in raw_ft if isinstance(ft, dict)]

    try:
        pos_prompt = build_positive_prompt(question, facets, taxonomies)
        pos_out = generate_text(pos_prompt, temperature=0.6, debug=True)
        positive_responses.append(pos_out)
    except torch.cuda.OutOfMemoryError:
        print(f"\n⚠️ OOM at row {idx}, switching to CPU temporarily...")
        model.to("cpu")
        pos_out = generate_text(pos_prompt, temperature=0.6, debug=False)
        positive_responses.append(pos_out)
        model.to(device)
    finally:
        torch.cuda.empty_cache()
        gc.collect()

    # --- Print summary per question ---
    print(f"\n🟩 Row {idx}")
    print(f"Q: {question}")
    print(f"Facets: {facets}")
    print(f"Taxonomies: {taxonomies}")
    print(f"✅ Clarifying Question: {pos_out}")
    print("------------------------------------------------------\n")

# ==============================================================
# 🔹 SAVE OUTPUT
# ==============================================================
df["positive_response"] = positive_responses
df.to_csv("positive_clarifying_responses.csv", index=False)
print("\n✅ Saved results to positive_clarifying_responses.csv")


Using device: cuda
Processing 500 rows


Generating positive clarifying questions:   0%|          | 0/500 [00:00<?, ?it/s]


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the original TV series premiere date or another related Simpson's media release?[/OUTPUT] 



Generating positive clarifying questions:   0%|          | 1/500 [00:29<4:03:08, 29.24s/it]


🟩 Row 0
Q: When did the simpsons first air on television?
Facets: []
Taxonomies: []
✅ Clarifying Question: Are you asking about the original TV series premiere date or another related Simpson's media release?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the federal legal age for marriage across all states, or is there a specific state you're interested in?[/OUTPUT] 



...


🟩 Row 492
Q: When did the baseball season go to 162 games?
Facets: []
Taxonomies: []
✅ Clarifying Question: Are you asking about when Major League Baseball adopted the 162-game schedule, or about a specific team's season length changing to 162 games?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]Are you asking about the total career home runs hit by a single player, or do you want information on the current record holder as of a specific date?[/OUTPUT] 



Generating positive clarifying questions:  99%|█████████▉| 496/500 [1:35:07<00:50, 12.71s/it]


🟩 Row 495
Q: Who has the most career homeruns in mlb?
Facets: []
Taxonomies: []
✅ Clarifying Question: Are you asking about the total career home runs hit by a single player, or do you want information on the current record holder as of a specific date?
------------------------------------------------------


=== RAW MODEL OUTPUT ===
 [OUTPUT]Could you specify the context or year you're asking about regarding the USS Maine explosion?[/OUTPUT] 



Generating positive clarifying questions: 100%|██████████| 500/500 [1:35:25<00:00, 11.45s/it]


🟩 Row 496
Q: Who was blamed for the explosion of the uss maine?
Facets: []
Taxonomies: []
✅ Clarifying Question: Could you specify the context or year you're asking about regarding the USS Maine explosion?
------------------------------------------------------


✅ Saved results to positive_clarifying_responses.csv


In [6]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
import pandas as pd

In [7]:
from peft import LoraConfig

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,                      # Rank of the low-rank matrices
    lora_alpha=32,             # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                    "gate_proj", "up_proj", "down_proj"],  # Modules to apply LoRA to
    lora_dropout=0.05,         # Dropout probability
    bias="none",               # Bias training strategy
    task_type="CAUSAL_LM"      # Task type
)

# Now create your trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=training_args,
    peft_config=lora_config,  # Now this will work
)

NameError: name 'model' is not defined